In [3]:
import wikipediaapi
import wikipedia
import requests
from bs4 import BeautifulSoup

# Wikipedia-Seitenname
page_title = "Liste_der_Mitglieder_des_Deutschen_Bundestages_(20._Wahlperiode)"

# Methode 1: Verwendung von `wikipedia-api`
def fetch_with_wikipedia_api(title):
    wiki_wiki = wikipediaapi.Wikipedia(user_agent='sophiarchy', language='de')
    page = wiki_wiki.page(title)
    
    if page.exists():
        print("=== Mit `wikipedia-api` ===")
        print("Seitentitel:", page.title)
        print("Kurzbeschreibung:", page.summary[:300])  # kurze Zusammenfassung
        print("Vollständiger Inhalt (erste 1000 Zeichen):\n", page.text[:1000])  # vollständiger Text
    else:
        print("Seite nicht gefunden (wikipedia-api)")

# Methode 2: Verwendung von `wikipedia`
def fetch_with_wikipedia_lib(title):
    wikipedia.set_lang("de")
    try:
        page = wikipedia.page(title)
        print("\n=== Mit `wikipedia` Bibliothek ===")
        print("Seitentitel:", page.title)
        print("Vollständiger Inhalt (erste 1000 Zeichen):\n", page.content[:1000])
    except wikipedia.exceptions.DisambiguationError as e:
        print("Mehrdeutigkeit gefunden:", e.options)
    except wikipedia.exceptions.PageError:
        print("Seite nicht gefunden (wikipedia Bibliothek)")

# Methode 3: Verwendung von `requests` und `BeautifulSoup`
def fetch_with_requests_and_bs(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    print("\n=== Mit `requests` und `BeautifulSoup` ===")
    print("Seitentitel:", soup.find('h1').text)
    
    # Beispiel: Nur die ersten Tabellen ausgeben
    tables = soup.find_all("table", {"class": "wikitable"})
    if tables:
        print("Anzahl gefundener Tabellen:", len(tables))
        print("Inhalt der ersten Tabelle (max. 1000 Zeichen):\n", tables[0].get_text(strip=True)[:1000])
    else:
        print("Keine Tabellen gefunden")

# Testen aller Methoden
fetch_with_wikipedia_api(page_title)
fetch_with_wikipedia_lib(page_title)
fetch_with_requests_and_bs("https://de.wikipedia.org/wiki/" + page_title)


=== Mit `wikipedia-api` ===
Seitentitel: Liste der Mitglieder des Deutschen Bundestages (20. Wahlperiode)
Kurzbeschreibung: Diese Liste gibt einen Überblick über alle Mitglieder des Deutschen Bundestages der 20. Wahlperiode nach der Bundestagswahl 2021. Die Wahlperiode begann mit der konstituierenden Sitzung am 26. Oktober 2021 und endet mit der konstituierenden Sitzung des 21. Deutschen Bundestages nach der nächsten Bun
Vollständiger Inhalt (erste 1000 Zeichen):
 Diese Liste gibt einen Überblick über alle Mitglieder des Deutschen Bundestages der 20. Wahlperiode nach der Bundestagswahl 2021. Die Wahlperiode begann mit der konstituierenden Sitzung am 26. Oktober 2021 und endet mit der konstituierenden Sitzung des 21. Deutschen Bundestages nach der nächsten Bundestagswahl (regulär 2025).
Von den ursprünglich 736 Sitzen im Bundestag entfielen auf Basis des Wahlergebnisses 206 auf die Sozialdemokratische Partei Deutschlands (SPD), 152 auf die Christlich Demokratische Union Deutschlands (CD

In [12]:
import requests
from bs4 import BeautifulSoup
import csv

# URL of the Wikipedia page
url = "https://de.wikipedia.org/wiki/Liste_der_Mitglieder_des_Deutschen_Bundestages_(20._Wahlperiode)"

# Retrieve the webpage
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Function to find the table under a specific heading
def get_table_under_heading(soup, heading_text):
    # Search for all headings and look for the specified heading text
    headings = soup.find_all(["h2", "h3"])  # Adjust depending on the actual heading structure
    
    for heading in headings:
        if heading_text in heading.get_text():
            # The next found "wikitable" after the heading
            next_table = heading.find_next("table", {"class": "wikitable"})
            return next_table
    return None

# Retrieve the table under the "Abgeordnete" heading
table = get_table_under_heading(soup, "Abgeordnete")

# Create and save the CSV file
csv_filename = "Abgeordnete_Bundestag_20_Wahlperiode.csv"
base_url = "https://de.wikipedia.org"  # Base URL for relative links
wahlperiode = 20  # Add the Wahlperiode value

if table:
    rows = table.find_all("tr")
    
    # Open the CSV file in write mode
    with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        
        # Write the header row (if available)
        header = ["Wahlperiode"]  # Set "Wahlperiode" as the first column
        header.extend([col.get_text(strip=True) for col in rows[0].find_all(["th", "td"])])
        header.append("Link")  # Add a column for Links
        writer.writerow(header)
        
        # Write data rows
        for row in rows[1:]:  # Skip the header row
            columns = row.find_all(["th", "td"])
            row_text = [col.get_text(strip=True) for col in columns]
            
            # Check if a suitable link is available in the second column (name field)
            name_cell = columns[1]
            link = "Kein Link verfügbar"
            if name_cell:
                # Find all links and select the first one that isn't an image link
                for a_tag in name_cell.find_all("a", href=True):
                    href = a_tag["href"]
                    if not href.endswith((".jpg", ".jpeg", ".png", ".svg")):
                        link = base_url + href
                        break
            
            # Prepend the Wahlperiode and append the link to each row
            row_text.insert(0, wahlperiode)  # Insert Wahlperiode as the first column
            row_text.append(link)  # Append link as the last column
            writer.writerow(row_text)
    
    print(f"The table has been successfully saved as '{csv_filename}'.")
else:
    print("The table under the heading 'Abgeordnete' was not found.")


The table has been successfully saved as 'Abgeordnete_Bundestag_20_Wahlperiode.csv'.
